In [1]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import pandas as pd
import datetime
import uuid
from tqdm import tqdm
from datetime import date
from selenium.webdriver.chrome.options import Options

In [2]:
main_url = 'http://cronica.diputados.gob.mx/DDebates/index.html'

In [3]:
import requests

In [4]:
r = requests.get(main_url)

<Response [200]>

In [2]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/85.0.4183.87/chromedriver] found in cache


In [7]:
driver.get(main_url)

In [8]:
def month_to_number(m):
    if m == 'ene':
        return(1)
    elif m == 'feb':
        return(2)
    elif m == 'mar':
        return(3)
    elif m == 'abr':
        return(4)
    elif m == 'may':
        return(5)
    elif m == 'jun':
        return(6)
    elif m == 'jul':
        return(7)
    elif m == 'ago':
        return(8)
    elif m == 'sep':
        return(9)
    elif m == 'oct':
        return(10)
    elif m == 'nov':
        return(11)
    elif m == 'dic':
        return(12)

def get_date_speech(year_session, link_speech, text_day):
    day_speech = re.findall('[0-9]+', text_day)[0]
    month_speech = month_to_number(link_speech.split('/')[-2:][0])
    return(datetime.date(int(year_session), int(month_speech), int(day_speech)))

def get_links_legislaturas(legislaturas):
    """
    Parametros:
    -----------
    legislatura: list of int numbers of legislatura 
    
    Returns:
    --------
    lista de diccionario con los a el htmls de los debates legislativos
    """
    all_data = []
    
    for legislatura in legislaturas:
        link_debates = 'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/index.html'
        
        driver.get(link_debates)
        
        links_sesiones = []
        type_sesion = []
        for h in driver.find_elements_by_css_selector('a'):
            links_sesiones.append(h.get_attribute('href'))
            type_sesion.append(re.sub('^[ ]+', '', h.text))
        
        
        for x, y in zip(links_sesiones, type_sesion):
            driver.get(x)
            
            for i in driver.find_elements_by_css_selector('a'):
                year_session = re.findall('[0-9]{4}', driver.find_element_by_css_selector('body > table:nth-child(3) > tbody > tr > td').text)[0]
                link_ = i.get_attribute('href')
                
                fecha_speech = get_date_speech(year_session, link_, i.text)
        
                temp = {}
                temp['id_debate'] = str(uuid.uuid3(uuid.NAMESPACE_URL,link_))
                temp['legislatura'] = legislatura
                temp['date_debate'] = fecha_speech
                temp['type_sesion'] = y
                temp['link_debate'] = link_
                all_data.append(temp)
    return(all_data)

In [9]:
r = requests.get(main_url)

In [10]:
legislatura = 63

In [46]:
r = requests.get('http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/index.html')

In [ ]:
r

In [47]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(r.content, 'html.parser')

In [49]:
driver.get('http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/' + soup.findAll('a')[0].get('href'))

In [27]:
soup = BeautifulSoup(r.content, 'html.parser')

In [34]:
len(soup.findAll('a'))

37

In [36]:
soup.findAll('a')[0].get('href')
http://cronica.diputados.gob.mx/DDebates/63/1er/1P/Ord/ago/01L63A1P101.html

'ago/01L63A1P101.html'

In [50]:
link_debate = 'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/index.html'

In [51]:
r = requests.get(link_debate)
soup = BeautifulSoup(r.content, 'html.parser')

In [54]:
h = soup.findAll('a')[0]

In [69]:
link_session = 'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/' + h.get('href')
urls = h.get('href')
reurls

'1er/1P/Ord/index.html'

In [59]:
r = requests.get(link_session)

In [62]:
soup = BeautifulSoup(r.content, 'html.parser')
re.findall('[0-9]{4}', soup.findAll('tbody')[1].find('tr').text)[0]

'2015'

In [65]:
i = soup.findAll('a')[0]

In [71]:
'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura)+'/'+ re.sub('index[.]html', '', urls) +i.get('href')

'http://cronica.diputados.gob.mx/DDebates/63/1er/1P/Ord/ago/01L63A1P101.html'

In [72]:
i.text

'\xa0\xa0Sábado 29\xa0\xa0'

In [74]:
def get_links_legislaturas(legislaturas):
    """
    Parametros:
    -----------
    legislatura: list of int numbers of legislatura 
    
    Returns:
    --------
    lista de diccionario con los a el htmls de los debates legislativos
    """
    all_data = []
    
    for legislatura in legislaturas:
        link_debates = 'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/index.html'
        
        r = requests.get(link_debates)
        soup = BeautifulSoup(r.content, 'html.parser')
        
        links_sesiones = []
        type_sesion = []
        urls = []
        for h in soup.findAll('a'):
            links_sesiones.append('http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/' + h.get('href'))
            type_sesion.append(re.sub('\xa0', '', h.text))
            urls.append(h.get('href'))
        
        
        for x, y, urls in zip(links_sesiones, type_sesion, urls):
            r = requests.get(x)
            soup = BeautifulSoup(r.content, 'html.parser')
            
            for i in soup.findAll('a'):
                year_session = re.findall('[0-9]{4}', soup.findAll('tbody')[1].find('tr').text)[0]
                link_ = 'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura)+'/'+ re.sub('index[.]html', '', urls) +i.get('href')
                
                fecha_speech = get_date_speech(year_session, link_, i.text)
        
                temp = {}
                temp['id_debate'] = str(uuid.uuid3(uuid.NAMESPACE_URL,link_))
                temp['legislatura'] = legislatura
                temp['date_debate'] = fecha_speech
                temp['type_sesion'] = y
                temp['link_debate'] = link_
                all_data.append(temp)
    return(all_data)

In [75]:
get_links_legislaturas([63])

[{'id_debate': 'c0ceba86-4f50-3a1f-8907-e83005ee8720',
  'legislatura': 63,
  'date_debate': datetime.date(2015, 8, 29),
  'type_sesion': 'Primer Periodo Ordinario',
  'link_debate': 'http://cronica.diputados.gob.mx/DDebates/63/1er/1P/Ord/ago/01L63A1P101.html'},
 {'id_debate': 'fb1799c9-00e9-3a85-b870-a54dbd24512a',
  'legislatura': 63,
  'date_debate': datetime.date(2015, 9, 1),
  'type_sesion': 'Primer Periodo Ordinario',
  'link_debate': 'http://cronica.diputados.gob.mx/DDebates/63/1er/1P/Ord/sep/01L63A1P102.html'},
 {'id_debate': '71979612-519d-32bd-ba03-5ce6c7b92e18',
  'legislatura': 63,
  'date_debate': datetime.date(2015, 9, 3),
  'type_sesion': 'Primer Periodo Ordinario',
  'link_debate': 'http://cronica.diputados.gob.mx/DDebates/63/1er/1P/Ord/sep/01L63A1P103.html'},
 {'id_debate': '37ea8705-c8f0-306d-91cf-6801203235a8',
  'legislatura': 63,
  'date_debate': datetime.date(2015, 9, 8),
  'type_sesion': 'Primer Periodo Ordinario',
  'link_debate': 'http://cronica.diputados.gob.m

In [17]:
htmls_speeches = get_links_legislaturas([63, 64])

In [19]:
df_debates = pd.DataFrame(htmls_speeches)

In [21]:
df_debates['counts'] = 1
df_debates[['legislatura', 'counts']].groupby(['legislatura']).count().reset_index()

,legislatura,counts
0,63,223
1,64,155


In [22]:
def obtain_texto_debate(driver):
    contenido_debate = driver.find_elements_by_class_name('Contenido')[0].text
    
    dict_contenidos = {}
    for i in contenido_debate.split('\n\n\n'):
        try:
            name_tema = re.sub('\n', '', re.findall('^[A-Z].*\n',i)[0])
            dict_contenidos[name_tema] = re.sub(name_tema, '', i)
        except:
            continue
    
    main_text = []
    for x in dict_contenidos.values():
        main_text.append(x)
    return(re.sub('\n', ' ', ' '.join(main_text)))

In [33]:

def obtain_text_sesion(driver):
    try:
        links_volumenes = []
        for i in driver.find_elements_by_css_selector('table')[1].find_elements_by_css_selector('a'):
            if any(re.findall('Volúmen' ,i.text)):
                links_volumenes.append(i.get_attribute('href'))
            else:
                continue
        
        driver.get(links_volumenes[0])
        return(obtain_texto_debate(driver))
    except:
        return(obtain_texto_debate(driver))
        
def obtain_texto_debate(driver):
    contenido_debate = driver.find_elements_by_class_name('Contenido')[0].text
    
    dict_contenidos = {}
    for i in contenido_debate.split('\n\n\n'):
        try:
            name_tema = re.sub('\n', '', re.findall('^[A-Z].*\n',i)[0])
            dict_contenidos[name_tema] = re.sub(name_tema, '', i)
        except:
            continue
    
    main_text = []
    for x in dict_contenidos.values():
        main_text.append(x)
    return(re.sub('\n', ' ', ' '.join(main_text)))            

In [34]:
text_congress = []
id_debates = []
for x, y in tqdm(zip(df_debates.link_debate, df_debates.id_debate), total = len(df_debates.link_debate)):
    driver.get(x)
    try:
        text_congress.append(obtain_text_sesion(driver))
        id_debates.append(y)
    except:
        print(i)

 45%|████▍     | 169/378 [08:20<06:36,  1.90s/it]  

http://cronica.diputados.gob.mx/DDebates/63/1er/1P/Ord/oct/Sesion13.html


 52%|█████▏    | 196/378 [10:17<04:29,  1.48s/it]  

http://cronica.diputados.gob.mx/DDebates/63/1er/1P/Ord/oct/Sesion13.html


 73%|███████▎  | 275/378 [13:55<01:08,  1.49it/s]  

http://cronica.diputados.gob.mx/DDebates/63/1er/1P/Ord/oct/Sesion13.html


100%|██████████| 378/378 [21:27<00:00,  3.40s/it]  


In [37]:
text_data = pd.DataFrame({'full_text_session':text_congress, 
             'id_debate':id_debates})

In [42]:
full_text_session = pd.merge(left = df_debates, 
         right = text_data, 
         on = ['id_debate'],
         how = 'left')

In [47]:
full_text_session[full_text_session['legislatura'] == 64][['id_debate', 'full_text_session']].to_csv('../../data/full_text_legislatura64.csv')

In [49]:
text_congress[10]

'  ASISTENCIA El Presidente diputado José de Jesús Zambrano Grijalva: Pido a la Secretaría que haga del conocimiento de esta Presidencia el resultado del cómputo de asistencia de diputadas y diputados. La Secretaria diputada Ana Guadalupe Perea Santos: Se informa a la Presidencia que existen registrados previamente 348 diputadas y diputados. Por lo tanto, hay quórum. El Presidente diputado José de Jesús Zambrano Grijalva(a las 10:28 horas): Se abre la sesión.  El Presidente diputado José de Jesús Zambrano Grijalva: Consulte la Secretaría a la asamblea si se dispensa la lectura al orden del día, en virtud de que se encuentra publicado en la Gaceta Parlamentaria. La Secretaria diputada Ana Guadalupe Perea Santos: Por instrucciones de la Presidencia, en votación económica, se consulta a la asamblea si se dispensa la lectura al orden del día. Las diputadas y los diputados que estén por la afirmativa sírvanse manifestarlo. Gracias. Las diputadas y los diputados que estén por la negativa sír

In [ ]:
full_text_session.to_csv('../../data/full_text_legislatura')

In [16]:
pd.DataFrame(htmls_speeches)

,id_debate,legislatura,date_debate,type_sesion,link_debate
0,c0ceba86-4f50-3a1f-8907-e83005ee8720,63,2015-08-29,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
1,fb1799c9-00e9-3a85-b870-a54dbd24512a,63,2015-09-01,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
2,71979612-519d-32bd-ba03-5ce6c7b92e18,63,2015-09-03,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
3,37ea8705-c8f0-306d-91cf-6801203235a8,63,2015-09-08,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
4,a530b0c6-94a2-3b22-955f-5cb57544dea5,63,2015-09-10,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
...,...,...,...,...,...
373,b3f2f4ad-b4cb-3ef7-8c27-304f1f4565c4,64,2020-03-19,Segundo Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...
374,7f1c9c76-f4aa-3772-a37c-95d627b805d4,64,2020-03-19,Segundo Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...
375,a36388e6-7e70-3379-afbd-7560939c6952,64,2020-06-30,Primer Periodo Extraordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...
376,d570d6a4-9083-3293-8d59-49f1939659dd,64,2020-07-22,Segundo Periodo Extraordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...


In [17]:
driver.get(pd.DataFrame(htmls_speeches).link_debate[0])

In [34]:
def metadata_debate(driver):
    director_general_cronica_parlamentaria = driver.find_element_by_id('T1').text.split('\n')[-1:][0]
    presidente = driver.find_element_by_id('T2').text.split('\n')[-1:][0]
    director_diario_de_los_debates = driver.find_element_by_id('T3').text.split('\n')[-1:][0]
    sesion = driver.find_element_by_id('Sesion').text
    return(director_general_cronica_parlamentaria, presidente, director_diario_de_los_debates, sesion)

In [26]:
driver.find_element_by_id('T3').text.split('\n')[-1:][0]

'Luis Alfredo Mora Villagómez'

In [36]:
metadata_debate(driver)

('Gilberto Becerril Olivares',
 'Diputado José de Jesús Zambrano Grijalva',
 'Luis Alfredo Mora Villagómez',
 'Sesión No. 1')

In [59]:
def obtain_entradas_debates(driver):
    contenido_debate = driver.find_elements_by_class_name('Contenido')[0].text
    
    dict_contenidos = {}
    for i in contenido_debate.split('\n\n\n'):
        name_tema = re.sub('\n', '', re.findall('^[A-Z].*\n',i)[0])
        dict_contenidos[name_tema] = re.sub(name_tema, '', i)
    return(dict_contenidos)